In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
from analysis_queries import run_analysis_queries
from cache_operations import cache_table, check_cached_tables, uncache_table
from parquet_operations import partition_and_create_temp_table
import os

# Set the working directory
os.chdir("C:\\Users\\Sebastian\\GitHub\\Home_Sales\\Starter_Code")

# Create a Spark session
spark = SparkSession.builder.appName("HomeSalesAnalysis").getOrCreate()

# Read the home_sales_revised.csv data into a Spark DataFrame
file_path = "path/to/home_sales_revised.csv"  # Replace with the actual path
home_sales_data = spark.read.format("csv").option("header", "true").load(file_path)

# Create a temporary table called home_sales
home_sales_data.createOrReplaceTempView("home_sales")

# Run analysis queries
run_analysis_queries(spark)

# Cache your temporary table home_sales
cache_table(spark, "home_sales")

# Check if your temporary table is cached
check_cached_tables(spark)

# Partition by the "date_built" field on the formatted parquet home sales data.
partitioned_file_path = "path/to/partitioned_home_sales.parquet"  # Replace with the actual path
partition_and_create_temp_table(spark, home_sales_data, partitioned_file_path)

# Uncache the home_sales temporary table.
uncache_table(spark, "home_sales")

# Stop the Spark session
spark.stop()


ModuleNotFoundError: No module named 'analysis_queries'

In [ ]:
def run_analysis_queries(spark):
    # Add the SparkSQL queries for analysis here
    # ...

    # Example:
    query = """
    SELECT year, AVG(price) as avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY year
    """
    result = spark.sql(query)
    result.show()


In [ ]:
def cache_table(spark, table_name):
    spark.sql(f"CACHE TABLE {table_name}")

def check_cached_tables(spark):
    cached_tables = spark.sql("SHOW TABLES")
    cached_tables.show()
    
def uncache_table(spark, table_name):
    spark.sql(f"UNCACHE TABLE {table_name}")


In [ ]:
def partition_and_create_temp_table(spark, data_frame, file_path):
    # Partition by the "date_built" field on the formatted parquet home sales data.
    data_frame.write.partitionBy("date_built").mode("overwrite").parquet(file_path)

    # Create a temporary table for the parquet data.
    parquet_data = spark.read.parquet(file_path)
    parquet_data.createOrReplaceTempView("parquet_home_sales")
